In [4]:

!pip install numpy scipy scikit-learn


from google.colab import drive
drive.mount('/content/drive')

import zipfile
import os

zip_path = '/content/drive/MyDrive/data.zip'
extract_dir = '/content/data'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


muse_zip_path = os.path.join(extract_dir, 'data', 'MuseData.zip')
muse_extract_path = os.path.join(extract_dir, 'MuseData')
with zipfile.ZipFile(muse_zip_path, 'r') as zip_ref:
    zip_ref.extractall(muse_extract_path)

Mounted at /content/drive


In [ ]:
import numpy as np
from scipy.signal import butter, filtfilt, resample
from scipy.interpolate import interp1d
from sklearn.preprocessing import MinMaxScaler
import os


fs = 256
lowcut = 5.0
highcut = 95.0
mu_path = os.path.join(muse_extract_path, 'MU.txt')


def bandpass_filter(signal, lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)


def interpolate_nans(signal):
    nans = np.isnan(signal)
    if np.any(nans):
        x = np.arange(len(signal))
        f = interp1d(x[~nans], signal[~nans], kind='linear', fill_value='extrapolate')
        signal[nans] = f(x[nans])
    return signal


signals = []

with open(mu_path, 'r') as f:
    for line in f:
        parts = line.strip().split('\t')
        raw_str = parts[6]
        signal = np.array([int(val) for val in raw_str.split(',')], dtype=np.float32)


        signal = bandpass_filter(signal, lowcut, highcut, fs)


        signals.append(signal)


lengths = [len(sig) for sig in signals]
median_len = int(np.median(lengths))

resampled_signals = [resample(sig, median_len) for sig in signals]


interpolated_signals = [interpolate_nans(sig) for sig in resampled_signals]


scaler = MinMaxScaler()
normalized_signals = []
for sig in interpolated_signals:
    sig = sig.reshape(-1, 1)
    norm = scaler.fit_transform(sig).flatten()
    normalized_signals.append(norm)

final_data = np.array(normalized_signals)
print("Final shape:", final_data.shape)
np.save('/content/muse_preprocessed_individual_signals.npy', final_data)
print("Saved ")
